# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Clustering_hack_data').getOrCreate()

In [2]:
from pyspark.ml.clustering import KMeans

data = spark.read.csv("hack_data.csv",header=True,inferSchema=True)

In [3]:
data.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)

In [4]:
data.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [25]:
data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [5]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [6]:
data.columns

['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'Location',
 'WPM_Typing_Speed']

In [9]:
col_new=data.select(['Session_Connection_Time',
 'Bytes Transferred',
 'Kali_Trace_Used',
 'Servers_Corrupted',
 'Pages_Corrupted',
 'WPM_Typing_Speed'])

In [12]:
data_new= col_new.na.drop()

In [13]:
vec_assembler = VectorAssembler(inputCols = data_new.columns, outputCol='features')

In [26]:
data_final = vec_assembler.transform(data_new)
data_final.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|
|                    2.0|           228.08|              1|             2.48|            8.0|            70.8|[2.0,228.08,1.0,2...|
|                   20.0|            408.5|              0|             3.57

In [36]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import VectorIndexer
feature_Indexer = VectorIndexer(inputCol="features", \
                               outputCol="features_index",\
                               maxCategories=4).fit(data_final)
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans() \
          .setK(2) \
          .setFeaturesCol("features_index")\
          .setPredictionCol("prediction")

# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[feature_Indexer, kmeans])

model = pipeline.fit(data_final.select("features"))

prediction = model.transform(data_final.select("features"))
prediction.show()

+--------------------+--------------------+----------+
|            features|      features_index|prediction|
+--------------------+--------------------+----------+
|[8.0,391.09,1.0,2...|[8.0,391.09,1.0,2...|         0|
|[20.0,720.99,0.0,...|[20.0,720.99,0.0,...|         1|
|[31.0,356.32,1.0,...|[31.0,356.32,1.0,...|         0|
|[2.0,228.08,1.0,2...|[2.0,228.08,1.0,2...|         0|
|[20.0,408.5,0.0,3...|[20.0,408.5,0.0,3...|         0|
|[1.0,390.69,1.0,2...|[1.0,390.69,1.0,2...|         0|
|[18.0,342.97,1.0,...|[18.0,342.97,1.0,...|         0|
|[22.0,101.61,1.0,...|[22.0,101.61,1.0,...|         0|
|[15.0,275.53,1.0,...|[15.0,275.53,1.0,...|         0|
|[12.0,424.83,1.0,...|[12.0,424.83,1.0,...|         0|
|[15.0,249.09,1.0,...|[15.0,249.09,1.0,...|         0|
|[32.0,242.48,0.0,...|[32.0,242.48,0.0,...|         0|
|[23.0,514.54,0.0,...|[23.0,514.54,0.0,...|         0|
|[9.0,284.77,0.0,3...|[9.0,284.77,0.0,3...|         0|
|[27.0,779.25,1.0,...|[27.0,779.25,1.0,...|         1|
|[12.0,307

In [38]:
prediction = model.transform(data_final)
prediction.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|      scaledFeatures|      features_index|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|[8.0,391.09,1.0,2...|         0|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|[20.0,720.99,0.0,...|         1|
|                   31.0|           356.32|              1|             3.7

In [39]:
from pyspark.ml.clustering import KMeans

kmeans = KMeans(k=2, seed=1)
model = kmeans.fit(data_final.select('features'))

In [41]:
predicted = model.transform(data_final)
predicted.show()

+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
|Session_Connection_Time|Bytes Transferred|Kali_Trace_Used|Servers_Corrupted|Pages_Corrupted|WPM_Typing_Speed|            features|      scaledFeatures|prediction|
+-----------------------+-----------------+---------------+-----------------+---------------+----------------+--------------------+--------------------+----------+
|                    8.0|           391.09|              1|             2.96|            7.0|           72.37|[8.0,391.09,1.0,2...|[0.56785108466505...|         1|
|                   20.0|           720.99|              0|             3.04|            9.0|           69.08|[20.0,720.99,0.0,...|[1.41962771166263...|         0|
|                   31.0|           356.32|              1|             3.71|            8.0|           70.58|[31.0,356.32,1.0,...|[2.20042295307707...|         1|
|               

In [42]:
predicted.head()

Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, WPM_Typing_Speed=72.37, features=DenseVector([8.0, 391.09, 1.0, 2.96, 7.0, 72.37]), scaledFeatures=DenseVector([0.5679, 1.3658, 1.9976, 1.2859, 2.2849, 5.3963]), prediction=1)